In [ ]:
from globvars import dir_fig, data_dir
import argparse
from os.path import join as pjoin


def sifnn(s,sep=','):
    if s is None:
        return None
    else:
        return s.split(sep)

parser = argparse.ArgumentParser()
parser.add_argument('--subjects')
parser.add_argument('--medconds')
parser.add_argument('--tasks')
parser.add_argument('--rawnames')
parser.add_argument('--use_data_afterICA',type=int)
parser.add_argument('--input_subdir')
parser.add_argument('--output_subdir')
parser.add_argument('--input_rawname_type')
parser.add_argument('--roi')

DEBUG = 1
if DEBUG:
    print("-------------------------------")
    print("--------DEBUG    --------------")
    print("-------------------------------")
    arglist = ['--rawnames','S01_off_move','--input_rawname_type', 'resample,notch',
              '--use_data_afterICA','0']
    args = parser.parse_args(arglist)
else:
    args = parser.parse_args()
    
    
rawnames = sifnn(args.rawnames)
if rawnames is None:
    rawnames = ["S01_off_hold"];
    
subjstrs = sifnn(args.subjects)
if subjstrs is None:
    subjstrs = ["S01", "S02", "S03", "S04", "S05", "S06", "S07", "S08", "S09", "S10" ];

medconds = sifnn(args.medconds)
if medconds is None:
    medconds = ["off", "on"];
    
tasks = sifnn(args.tasks)
if tasks is None:
    tasks = ["hold", "move"];
    
input_rawname_type = sifnn(args.input_rawname_type)
if input_rawname_type is None:
    input_rawname_type = ["resample", "notch", "highpass"]; # default
# %input_rawname_type = ["resample", "afterICA"]
# %input_rawname_type = ["SSS", "notch", "highpass", "resample", "afterICA"]
    
input_subdir = args.input_subdir
if input_subdir is None:
    input_subdir = "";

output_subdir = args.output_subdir
if output_subdir is None:
    output_subdir = "";

use_data_afterICA = args.use_data_afterICA
if use_data_afterICA is None:
    use_data_afterICA = 1;
    
roi = args.roi
if roi is None:
    roi = "parcel_aal_surf";
    

if use_data_afterICA:
  input_rawname_type = input_rawname_type + [ "afterICA"]
rawname_suffix = '';
# for i in range(len(input_rawname_type) ):
#   rawname_suffix = "%s_%s" % (rawname_suffix,input_rawname_type[i] );
# rawname_suffix = "%s_%s" % (rawname_suffix,"raw" );
rawname_suffix = '_'.join( input_rawname_type + ['raw'] )

resname_add_str = '';
# %if any(contains(input_rawname_type, "SSS" ) ) 
# %  if any(strcmp(input_rawname_type, "afterICA" ) )   
if rawname_suffix.find("SSS") >= 0:
  remove_bad_channels = 0;
#   % I better control it with setting subfolder names
#   %resname_add_str = "SSS"
#   %if contains(rawname_suffix, "afterICA")
#   %  resname_add_str = strcat(resname_add_str, "_afterICA")
#   %end
else:
 remove_bad_channels = 1
#end

# % Brodmann 4 -- M1,   Brodmann 6 -- PMC
# %roi = {"Brodmann area 4"};
# %roi = {"Brodmann area 6"};
# %roi = {"HirschPt2011,2013"}
# %roi = {"HirschPt2011,2013","Thalamus"}
# %
# %roi = {"Brodmann area 4","Brodmann area 6"};
# %roi = {"HirschPt2011"};
save_srcrec          = 1;
if DEBUG:
  save_srcrec          = 0;

use_DICS = 0;

import pymatreader as pymr
#% Load Jan-provided info
#if os.path.exists("Info.mat"):
info = pymr.read_mat(pjoin(data_dir,"Info.mat"))


highpass_freq = 1.5;

#f_scalemat = pymr.read_mat(pjoin(code_dir,('head_scalemat.mat') ) );
#some info about "spacing" parameter
#https://mne.tools/stable/overview/cookbook.html#setting-up-source-space
#add_dist = Add distance and patch information to the source space
#add_dist Can also be ‘patch’ to only compute patch information (requires SciPy 1.3+).

#https://mne.tools/stable/generated/mne.setup_source_space.html#mne.setup_source_space

import mne

In [ ]:
help(datetime)

In [ ]:
mtime

In [ ]:
from datetime import datetime

rawname = rawnames[0]

fname_raw_full = pjoin(data_dir, f'{rawname}_{rawname_suffix}.fif' )
assert os.path.exists(fname_raw_full)
mtime = datetime.fromtimestamp( os.stat(fname_raw_full).st_mtime ) 
display(mtime)
raw = mne.io.read_raw(fname_raw_full)

raw_grad = raw.copy()
raw_mag  = raw.copy()
megtype2subraw = {'grad':raw_grad, 'mag':raw_mag}
for megtype,subraw in megtype2subraw.items():
    megtype2subraw[megtype] = subraw.pick_types(megtype)

## TODO: read MEG artifacts here
## TODO maybe -- use code from collect artifacts to reject everything non-rest

# from the documentation example values
reject = dict(grad=4000e-13, # T / m (gradiometers)
              mag=4e-12, # T (magnetometers)                           
              )
#eeg=40e-6, # V (EEG channels)
#eog=250e-6 # V (EOG channels)

megtype2data_cov = {}
for megtype,subraw in megtype2subraw.items():
    reject_sub = {}
    reject_sub[megtype] = reject[megtype]
    
    # bad channels count here a lot
    # also does rescaling according to hardcorded values, so it is necessary 
    # that it was not recaled before
    data_cov = mne.compute_raw_covariance(subraw, picks='meg', tmin=0, tmax=None, 
                               reject_by_annotation=True, n_jobs=-1, reject=reject_sub)
    megtype2data_cov[megtype] = data_cov

## BEM

In [ ]:
subject = rawname.split('_')[0]; subject
subjects_dir = pjoin(data_dir,'subjects')

In [ ]:
fname_full_trans = pjoin(subjects_dir,f'{subject}_transform.fif')
trans = mne.read_trans(fname_full_trans)

fname_full_bem = pjoin(data_dir,f'{subject}-bem.fif')
load_bem = 1

if os.path.exists(fname_full_bem) and load_bem:
    bem = mne.read_bem_solution(fname_full_bem)
else:
    conductivity = (0.3,)  # for single layer
    # conductivity = (0.3, 0.006, 0.3)  # for three layers
    model = mne.make_bem_model(subject=subject, ico=4,
                               conductivity=conductivity,
                               subjects_dir=subjects_dir)
    bem = mne.make_bem_solution(model)
    #########


    mne.write_bem_solution(fname_full_bem,bem)

In [ ]:
help(mne.setup_source_space)

## SRC

In [ ]:
import gc; gc.collect()

In [ ]:
load_src = 1
fname_full_src_space = pjoin(data_dir,f'{subject}-src.fif')
fname_full_src_space_noCB = pjoin(data_dir,f'{subject}_noCB-src.fif')
fname_full_src_space_CB = pjoin(data_dir,f'{subject}_CB-src.fif')
CB_spacing = 12 #in mm

if os.path.exists(fname_full_src_space) and load_src:
    print('Loading source spaces')
#source_space_noCB = mne.read_source_spaces(fname_full_src_space)
    source_space_noCB = mne.read_source_spaces(fname_full_src_space_noCB)
    source_space_CB   = mne.read_source_spaces(fname_full_src_space_CB)
    source_space = source_space_noCB + source_space_CB
else:
### recalc
    # subject should correspond to freesurfer fiels
    source_space_noCB = mne.setup_source_space(subject, spacing='oct4', add_dist=True,
                                 subjects_dir=subjects_dir, n_jobs = -1)

    #around 258 for each hemi    
    mne.write_source_spaces(fname_full_src_space_noCB ,source_space_noCB,overwrite=True)

    
    # mindist : float
    #     Exclude points closer than this distance (mm) to the bounding surface.
    # exclude : float
    #     Exclude points closer than this distance (mm) from the center of mass
    #     of the bounding surface.

    volume_label = 'Left-Cerebellum-Cortex'
    #sphere = (0, 0, 0, 0.12)
    lh_cereb = mne.setup_volume_source_space(
        subject,  volume_label=volume_label, bem=bem, 
        subjects_dir=subjects_dir, pos=CB_spacing)

    volume_label = 'Right-Cerebellum-Cortex'
    rh_cereb = mne.setup_volume_source_space(
        subject,  volume_label=volume_label, bem=bem, 
        subjects_dir=subjects_dir, pos=CB_spacing)
    #mri=aseg_fname,exclude=10
    
    source_space_CB = lh_cereb + rh_cereb
    mne.write_source_spaces(fname_full_src_space_CB ,source_space_CB,overwrite=True)

    # Combine the source spaces
    #source_space_ext_left = source_space[0] + lh_cereb
    source_space = source_space_noCB + lh_cereb + rh_cereb
    mne.write_source_spaces(fname_full_src_space, source_space, overwrite=True)

srcd = {'cortex':source_space_noCB, 'CB':source_space_CB}
#srcd_sided = {'cortex_left':source_space_noCB, 'CB':source_space_CB}

In [ ]:
source_space

In [ ]:
mne.viz.plot_bem(subject, subjects_dir=subjects_dir, src=source_space)

In [ ]:
source_space[0]['type'], source_space[0]['subject_his_id'], \
    source_space[0]['coord_frame'], source_space[0]['id']

In [ ]:
# fig = mne.viz.plot_alignment(subject=subject, subjects_dir=subjects_dir,
#                              surfaces='white', coord_frame='mri',
#                              src=source_space)
# mne.viz.set_3d_view(fig, azimuth=173.78, elevation=101.75,
#                     distance=0.30, focalpoint=(-0.03, -0.01, 0.03))

In [ ]:
#write_source_spaces('sample-oct6-src.fif', src)

In [ ]:
from mne.io.pick import channel_type
[channel_type(raw.info, ind) for ind in range(len(raw.info.ch_names) )]

In [ ]:
from mne.channels.channels import  _contains_ch_type
_contains_ch_type(raw.info, 'grad' )

## Forward solution

In [ ]:
101 lh
102 rh

In [ ]:
src['id'],src['seg_name']

In [ ]:
recalc_forward_solution = 1
if not recalc_forward_solution:
    sys.exit(0)
# we need to do separate forward solutions for each of the source spaces
for megtype,subraw in megtype2subraw.items():
    for src_type,src in srcd.items():
        # head<->MRI transform *-trans.fif file produced during coregistration. 
        fwd = mne.make_forward_solution(subraw.info, trans=trans, src=src, 
                                        bem=bem,
                                        meg=True, eeg=False, mindist=5.0, n_jobs=-1,
                                        verbose=0)
        print(fwd)
        #leadfield = fwd['sol']['data']



        fname_full_fwd = pjoin(data_dir,f'{rawname}_{src_type}_{megtype}-fwd.fif')
        mne.write_forward_solution(fname_full_fwd,fwd,overwrite=1)

## Inverse solution

In [ ]:
megtype2subraw.keys()

In [ ]:
recalc_lcmv = 1

from mne.beamformer import make_lcmv, apply_lcmv, apply_lcmv_raw
noise_cov = None
depth = 0.8  # val from MNE doc
# bads are not used here
# gives error: Source reconstruction with several sensor types 
# requires a noise covariance matrix to be able to apply whitening
megtype2filters = {}
for megtype,subraw in megtype2subraw.items():
    megtype2filters[megtype] = {}
    for src_type,src in srcd.items():
        fname_full_inv = pjoin(data_dir,f'{rawname}_{src_type}_{megtype}-lcmv.h5')
        if not recalc_lcmv:
            filters = mne.beamformer.read_beamformer(fname_full_inv)
        else:

            fname_full_fwd = pjoin(data_dir,f'{rawname}_{src_type}_{megtype}-fwd.fif')
        #fname_full_fwd = pjoin(data_dir,f'{rawname}_{megtype}-fwd.fif')
            forward = mne.read_forward_solution(fname_full_fwd)

            filters = make_lcmv(subraw.info, forward, megtype2data_cov[megtype],
                                reg=0.05,
                                noise_cov=noise_cov, pick_ori='max-power',
                                weight_norm='unit-noise-gain', rank=None, depth=depth)        
            filters.save(fname_full_inv, overwrite=True)
        megtype2filters[megtype][src_type] = filters
    
megtype2stc = {}
for megtype,subraw in megtype2subraw.items():
    megtype2stc[megtype] = {}
    for src_type,src in srcd.items():
        stc = apply_lcmv_raw(subraw, megtype2filters[megtype][src_type])
        megtype2stc[megtype][src_type] = stc

### plot inverse solution

In [ ]:
lims = [0.3, 0.45, 0.6]
kwargs = dict(src=src, subject=subject, subjects_dir=subjects_dir,
              initial_time=0.087, verbose=True)
#clim=dict(kind='value', pos_lims=lims),
stc.plot( **kwargs)
#mode='stat_map', 

In [ ]:
stc.plot(mode='glass_brain', clim=dict(kind='value', lims=lims), **kwargs)

In [ ]:
brain = stc_vec.plot_3d(
    clim=dict(kind='value', lims=lims), hemi='both', size=(600, 600),
    views=['sagittal'],
    # Could do this for a 3-panel figure:
    # view_layout='horizontal', views=['coronal', 'sagittal', 'axial'],
    brain_kwargs=dict(silhouette=True),
    **kwargs)

### Morph

In [ ]:
#stc[0]

In [ ]:
help(morph.save)

In [ ]:
morphd = {}
for src_type,stc in d.items():
    #stc = megtype2stc['mag']['cortex']
    if src_type == 'cortex':
        morph = mne.compute_source_morph(src_type, subject_from=subject,                        
                             subjects_dir=subjects_dir, verbose=True)

    else:
        morph = mne.compute_source_morph(srcd[src_type], 
                subject_from=subject, subjects_dir=subjects_dir,
        niter_affine=[10, 10, 5], niter_sdr=[10, 10, 5],  # just for speed
        verbose=True)

    fname_full_morph = pjoin(data_dir, f'{subject}_{src_type}-morph.h5' )
    morph.save(fname_full_morph)
    morphd[src_type] = morph

In [ ]:
mne.

In [ ]:
#subject_to='fsaverage', src_to=src_to,
megtype2stc_fsav = {}
for megtype,d in megtype2stc.items():
    megtype2stc_fsav[megtype] = {}
    for src_type,stc in d.items():
        fname_full_morph = pjoin(data_dir, f'{subject}_{src_type}-morph.h5' )
        #stc = megtype2stc['mag']['cortex']
#         if src_type == 'cortex':
#             morph = mne.compute_source_morph(stc, subject_from=subject,                        
#                                  subjects_dir=subjects_dir, verbose=True)
            
#         else:
#             morph = mne.compute_source_morph(srcd[src_type], 
#                     subject_from=subject, subjects_dir=subjects_dir,
#             niter_affine=[10, 10, 5], niter_sdr=[10, 10, 5],  # just for speed
#             verbose=True)
            
#         morph.save(pjoin(data_dir, f'{subject}_{src_type}'))
        
        morph = morphd[src_type]
        stc_fsaverage = morph.apply(stc)
        megtype2stc_fsav[megtype][src_type] = stc_fsaverage
#src must be an instance of source space or surface source estimate, got <class 'mne.source_estimate.MixedSourceEstimate'> instead.


## Working with parcels

In [ ]:
labels = mne.read_labels_from_annot(
    'fsaverage', 'HCPMMP1_combined', 'lh', subjects_dir=subjects_dir)
len(labels)

In [ ]:
labels[0].subject

In [ ]:
labels_arg = labels # (aal_path,  atlas.labels[:3] )
stc = megtype2stc_fsav['mag']['cortex']
tca = mne.extract_label_time_course([stc], labels_arg,srcd['cortex'], 
                          mode = 'mean', allow_empty=False, verbose=10)

In [ ]:
data_path

In [ ]:
def inds_to_coords(i, j, k, affine):
    """
    Return X, Y, Z coordinates for i, j, k
    Parameters
    ----------
    i, j, k : int
        indices for aal_img.get_fdata()
    affine : np.array of shape (4, 4)
        The affine transformation from indices to coordinates
        given by nibabel.nifti1.Nifti1Image.affine
    Returns
    -------
    coords : np.array of shape (3, )
        The indices i, j, k transformed to coordinates in mm.
    References
    ----------
    https://nipy.org/nibabel/coordinate_systems.html#applying-the-affine
    """

    M = affine[:3, :3]
    abc = affine[:3, 3]

    return M.dot([i, j, k]) + abc

In [ ]:
import nibabel as nib
from nilearn.datasets import fetch_atlas_aal
mne_sample_data_path = mne.datasets.sample.data_path()
atlas = fetch_atlas_aal(data_dir=mne_sample_data_path)
maps = 'maps'
atlas_img = nib.load(atlas[maps])

value_to_label_dict = dict()
for idx, label in enumerate(atlas['labels']):
    value_to_label_dict[int(atlas['indices'][idx])] = label

value_to_label_dict['0'] = 'Background'

atlas_data = atlas_img.get_fdata()
affine = atlas_img.affine

mni_inuse = np.where(atlas_data > 0)
# get for mni_inuse the label indices
mni_label_inds = atlas_data[mni_inuse]

print(atlas_img.affine.shape, atlas_img.header.get_xyzt_units())

In [ ]:
value_to_label_dict

In [ ]:
xmax, ymax, zmax = atlas_data.shape

coords_shape = list(atlas_data.shape)
coords_shape.append(3)
atlas_coords = np.empty(coords_shape)

for idx_x in range(xmax):
    for idx_y in range(ymax):
        for idx_z in range(zmax):
            atlas_coords[idx_x, idx_y, idx_z] = \
                inds_to_coords(idx_x, idx_y, idx_z, affine)

In [ ]:
#mni_label_inds = mni_label_inds.astype(int)
atlas_data_i = atlas_data.astype(int)
indsu = np.unique(atlas_data_i)

d = {}
for a in indsu:
    indsofinds = np.where(atlas_data_i == a)
    d[a] = indsofinds    
    
# atlas_coords[d[0]].shape #(717224, 3)
# atlas_coords.shape #(91, 109, 91, 3)

In [ ]:
lbls = []
sidedict = {'L':'lh', 'R':'rh'}
for a in indsu:
    if a == 0:
        continue
    pos = atlas_coords[d[a]] / 1000.
    label = value_to_label_dict[a]
    print(label)
    side = sidedict.get(label[-1],None)
    if side is None:
        continue
    
    lbl = mne.Label(vertices=np.arange(len(pos) ), pos=pos, name=label,
              subject=subject, hemi=side)
    lbls.append(lbl)
    
    #vertices array, shape (N,)
    #    Vertex indices (0 based).


    #vertices=(), pos=None, values=None, hemi=None, comment='', 
    # name=None, filename=None, subject=None, color=None, *, verbose=None

    # verticesarray, shape (N,)

    #     Vertex indices (0 based).
    # posarray, shape (N, 3) | None

    #     Locations in meters. If None, then zeros are used.


In [ ]:
source_space[0].keys()

In [ ]:
source_space[0]['inuse']

In [ ]:
# Q: is there a canonical space MNE refers points to?
# Q: what are the labels one can use?

In [ ]:
source_space.in_label()
#Get a source estimate object restricted to a label.

In [ ]:
label_src_vertno_sel(label, src):
    """Find vertex numbers and indices from label.


In [ ]:
from mne import extract_label_time_course
extract_label_time_course(stcs, lbls, src, mode='auto', allow_empty=False, 
                              return_generator=False, *, mri_resolution=True,
                          verbose=None)

In [ ]:
parc = 'aparc' 
labels_parc = mne.read_labels_from_annot(
    subject, parc=parc, subjects_dir=subjects_dir)

In [ ]:
source_space

In [ ]:
len(['Frontal_Sup_Orb_L', 'Frontal_Mid_Orb_L', 
     'Frontal_Inf_Oper_L', 'Rolandic_Oper_L', 
     'Olfactory_L', 'Frontal_Med_Orb_L', 'Rectus_L', 
     'Cingulum_Post_L', 'Hippocampus_L', 'ParaHippocampal_L',
     'Amygdala_L', 'Amygdala_R', 'Occipital_Inf_L', 
     'Angular_L', 'Caudate_L', 'Putamen_L', 'Pallidum_L', 
     'Pallidum_R', 'Thalamus_L', 'Heschl_L', 'Heschl_R', 
     'Temporal_Pole_Mid_L', 
     'Cerebelum_3_L', 'Cerebelum_3_R', 'Cerebelum_4_5_L',
     'Cerebelum_7b_L', 'Cerebelum_9_L', 'Cerebelum_10_L', 'Cerebelum_10_R'])

In [ ]:
len(lbls)

In [ ]:
aal_path = pjoin(mne_sample_data_path, 'aal_SPM12/aal/atlas/AAL.nii')

#labels_arg -- a two-element list or tuple, the first element being a path to an atlas, 
# and the second being a list or dict of volume_labels to extract 
# (see mne.setup_volume_source_space() for details).

# returns   label_tcarray | list (or generator) of array, shape (n_labels[, n_orient], n_times)
#  Extracted time course for each label and source estimate.


labels_arg = lbls # (aal_path,  atlas.labels[:3] )
stc = megtype2stc['mag']
tca = mne.extract_label_time_course([stc], labels_arg,source_space, 
                          mode = 'mean', allow_empty=False, verbose=10)

In [ ]:
#mne.extract_label_time_course

In [ ]:
# mne.datasets.fetch_hcp_mmp_parcellation(subjects_dir=subjects_dir,
#                                         verbose=True)

# mne.datasets.fetch_aparc_sub_parcellation(subjects_dir=subjects_dir,
#                                           verbose=True)

labels = mne.read_labels_from_annot(
    'fsaverage', 'HCPMMP1', 'lh', subjects_dir=subjects_dir)


In [ ]:
labels = mne.read_labels_from_annot(
    subject, 'aparc', 'lh', subjects_dir=subjects_dir)
# No such file /home/demitau/data/subjects/S01/label/lh.HCPMMP1.annot, candidate parcellations in that directory:
# aparc
# aparc.a2009s
# aparc.DKTatlas
# BA_exvivo
# BA_exvivo.thresh
# mpm.vpnl

In [ ]:
labels

In [ ]:
len(labels)

In [ ]:
labels = mne.read_labels_from_annot(
    'fsaverage', 'HCPMMP1_combined', 'lh', subjects_dir=subjects_dir)
len(labels)

In [ ]:
labels[1] + labels[2]

In [ ]:
labels

In [ ]:
# morph

In [ ]:
# fetch_fsaverage(subjects_dir)  # ensure fsaverage src exists
# fname_fs_src = subjects_dir / 'fsaverage' / 'bem' / 'fsaverage-vol-5-src.fif'

# src_fs = mne.read_source_spaces(fname_fs_src)
# morph = mne.compute_source_morph(
#     src, subject_from='sample', src_to=src_fs, subjects_dir=subjects_dir,
#     niter_sdr=[5, 5, 2], niter_affine=[5, 5, 2], zooms=7,  # just for speed
#     verbose=True)
# stc_fs = morph.apply(stc)
# del stc

# stc_fs.plot(
#     src=src_fs, mode='stat_map', initial_time=0.085, subjects_dir=subjects_dir,
#     clim=dict(kind='value', pos_lims=lims), verbose=True)

# Matlab code

In [ ]:
atlas_type = "aal"
if strcmp(atlas_type, "aal")
  aal = ft_read_atlas('~/soft/fieldtrip/template/atlas/aal/ROI_MNI_V4.nii');
  atlas = aal;
elseif strcmp(atlas_type, "afni")  % has Brodmann areas, no sides
  afni = ft_read_atlas('~/soft/fieldtrip/template/atlas/afni/TTatlas+tlrc.HEAD');  % goes deeper in the sulcus
  atlas = afni;
end
%singleshell = load('~/soft/fieldtrip/template/headmodel/standard_singleshell');
source_grid = load('~/soft/fieldtrip/template/sourcemodel/standard_sourcemodel3d5mm');
source_grid.sourcemodel.coordsys = 'mni';
% pts_converted = mni2icbm_spm( pts )
% atlas.  pts_converted = mni2icbm_spm( pts )
atlas = ft_convert_units(atlas,'cm'); % ftrop and our sourcemodels have cm units
atlas.coordsys = 'mni';

% if roi is found in the atlas. Here I assume that if one is from the atlas than other rois too
if isfield(atlas, "tissuelabel")
  tlarr = atlas.tissuelabel;
elseif isfield(atlas, "brick0label")  
  tlarr = cat ( 1, {atlas.brick0label{:},atlas.brick1label{:} } );
end

read_upd_bads = 0;

%TODO: first read and do reject artifact, then append data then call source reconstruction
%cfg = [];
%data_merged =  ft_appenddata(cfg, data1_resampled, data2);

num_prcessed = 0;
for rawi = 1:length(rawnames)
  r = split(rawnames(rawi),'_');
  subjstr = r(1); medstr=r(2); taskstr=r(3);

%for subji = 1:length(subjstrs)
%  subjstr = subjstrs(subji);
  fprintf(" current subjstr=%s\n",subjstr)

  basename_head = sprintf('/headmodel_grid_%s.mat',subjstr);
  fname_head = strcat(data_dir, basename_head );
  hdmf = load(fname_head);   %hdmf.hdm, hdmf.mni_aligned_grid

%  for medcondi = 1:length(medconds)
%    medstr = medconds(medcondi);
%    for taski = 1:length(tasks)
%      taskstr = tasks(taski);

      % rawnames_suffix starts with '_'
      basename = sprintf('/%s_%s_%s%s.fif',subjstr,medstr,taskstr,rawname_suffix);

      %if use_data_afterICA
      %  basename = sprintf('/%s_%s_%s_resample_afterICA_raw.fif',subjstr,medstr,taskstr);
      %else
      %  % resample was notched but not highpassed
      %  %basename = sprintf('/%s_%s_%s_resample_raw.fif',subjstr,medstr,taskstr);
      %  basename = sprintf('/%s_%s_%s_resample_notch_highpass_raw.fif',subjstr,medstr,taskstr);
      %end
      fprintf(" Using basename=%s\n",basename)
      %basename = sprintf('/%s_%s_%s_resample_maxwell_raw.fif',subjstr,medstr,taskstr);
      fname = fullfile(data_dir, input_subdir, basename );
      if isfile(fname)   % if file exists
        fprintf("%s\n",fname)

        cfgload = [];
        cfgload.dataset = char(fname);          % fname should be char array (single quotes), not string (double quotes)
        cfgload.chantype = {'meg'};
        cfgload.coilaccuracy = 1;   % 0 1 or 2, idk what each of them means -- it is about combining grad and mag

        %cfgload.hpfilter = 'yes';
        %cfgload.hpfreq = highpass_freq;  % NO, I don't want to do it in FT because it will not take care of artifacts

        datall_ = ft_preprocessing(cfgload);   

        bads = {};
        if isfield(Info, subjstr)
          if isfield(Info.(subjstr).bad_channels, medstr) &&  isfield(Info.(subjstr).bad_channels.(medstr), taskstr)
            bads = Info.(subjstr).bad_channels.(medstr).(taskstr);
          end

          if read_upd_bads;
              fname_bads_mat = sprintf('%s_MEGch_bads_upd.mat',fname_noext);
              fname_bads_mat_full = fullfile( data_dir, fname_bads_mat);
              bads_upd = load(fname_bads_mat_full).bads;
              bads = bads_upd;
          end
        else
          fprintf('!!!! Info.mat does not contain %s field, assuming no bad channels',subjstr)
        end

        chanselarr = {'meg'};
        if remove_bad_channels && length(bads) > 0
          for chani = 1:length(bads)
            chanselarr{chani + 1} = sprintf('-%s',bads{chani});   % marks channel for removal from seelction
          end
        end

        selchan = ft_channelselection(chanselarr, datall_.label);
        bads = {};
        % I could also do ft_repairchannel
        
        cfgsel = [];
        cfgsel.channel = selchan;
        datall  = ft_selectdata(cfgsel, datall_);

        %else
        %  datall  = datall_;
        %end

        %if use_data_afterICA  % because we also do SSS which restores bad channels
        %  bads = {}
        %end
        %else
        %  datall  = datall_;
        %end


        %deal with artifacts
        %fname_srcrec_exclude = sprintf('/%s_%s_%s_ann_MEGartif.txt', subjstr,medstr,taskstr);
        fname_srcrec_exclude = sprintf('/%s_%s_%s_ann_srcrec_exclude.txt', subjstr,medstr,taskstr);
        filename      = fullfile(data_dir,fname_srcrec_exclude);
        FID = fopen(filename);
        fgets(FID); fgets(FID); % skip first 2 lines
        form='%f,%f,%s'; % we have 7 columns, then use 7 %f
        out = textscan(FID, form);
        times_artif = [ out{1}, out{1}+out{2} ];  %third column is interval type, which I don't need
        bins_artif = datall_.fsample * times_artif;  
        bins_artif = 1 + round(bins_artif);
        %bins_artif = times_artif;
        fclose(FID);


        cfg_ar = [];
        cfg_ar.artfctdef.imported.artifact = bins_artif;
        %cfg_ar.artfctdef.muscle.artifact = bins_artif;

        cfg_ar.artfctdef.reject          = 'partial';
        %cfg_ar.artfctdef.reject          = 'nan';
        %cfg_ar.artfctdef.reject          = 'value';
        %cfg_ar.artfctdef.value           = 1e-2

        cfg_ar.artfctdef.minaccepttim    = 1; %min length of remaining trial in seconds
        %   cfg.artfctdef.feedback        = 'yes' or 'no' (default = 'no')
        %   cfg.artfctdef.invert          = 'yes' or 'no' (default = 'no') -- invert artifact selection
        
        % unfortunately it cannot reduce trial size, just throw away it 
        % completely, so I do some dirty work later        
        data_cleaned = ft_rejectartifact(cfg_ar,datall) 
        %return

        for roii = 1:length(roi) 
          roicur = roi{roii};

          % scaling matrix, no longer used, kept for compatibility
          if isKey(f_scalemat.scalemat,subjstr)
            S = f_scalemat.scalemat(subjstr);
          else
            S = eye(3);
          end

          % do we find roi in the atlas?
          roimask = contains(cellstr(tlarr), roicur );
          if sum(roimask) == 1 
            cfg_vlookup = [];
            cfg_vlookup.atlas = atlas;
            cfg_vlookup.roi = roi;
            %cfg_vlookup.roi = atlas.tissuelabel; % this would lookup for all possible
            cfg_vlookup.inputcoord = 'mni';  % coord of the source
            %cfg_vlookup.inputcoord = 'tal';
            source_grid_mask = ft_volumelookup(cfg_vlookup,source_grid.sourcemodel);  % selecting only a subset
            fprintf("%s: Num of sources = %d\n",roicur,sum(source_grid_mask) )
          else
            fprintf("%s not found in atlas\n",roicur )
            source_grid_mask = [];
          end

          source_data = srcrec(subjstr,datall,data_cleaned,hdmf,{roicur},bads,S,source_grid,source_grid_mask,use_DICS);

          for fbi = 1:length(source_data) 
            source_data{fbi}.source_data.roi = {roicur};
          end

          if save_srcrec
            basename_srcd = sprintf("/srcd_%s_%s_%s_%s%s.mat",subjstr,medstr,taskstr,roicur,resname_add_str);
            %data_dir_out = strcat(
            fname_srcd = fullfile(data_dir, output_subdir, basename_srcd );
            fprintf("Saving to %s\n",fname_srcd)
            save(fname_srcd,"source_data","-v7.3");
          end
        end
      else
        fprintf(" file not found! %s",fname)
      end
%    end
%  end
 
  num_prcessed = num_prcessed + 1;
end

if num_prcessed == 0
  fprintf('AAAAAA')
end